In [7]:
library(stringr)
library(readr)
library(tibble)
library(psychometric)
library(dplyr)
library(purrr)

In [2]:
fit_glm <- function(data_df, formula_str, family){
    glm(
        stats::as.formula(formula_str),
        family = family,
        data = data_df
    )
}

In [3]:
fit_to_df <- function(fit){
    fit_df <- summary(fit)$coeff %>%
    as.data.frame() %>% rownames_to_column("variable")
    colnames(fit_df) <- c("variable", "estimate", "SE", "z_or_t_value", "P")
    fit_df
}


In [4]:
glm_fit_to_R2 <- function(glm_fit) {
    with(summary(glm_fit), 1 - deviance / null.deviance)
}

In [5]:
compose_regression_formula_str <- function(
    response, predictors, quote_char="`"
) {
    return(sprintf(
        "%s ~ 1 + %s",
        paste0(quote_char, response, quote_char),
        paste(sapply(
            predictors,
            function(term){paste0(quote_char, term, quote_char)}
        ), collapse = " + ")
    ))
}

In [6]:
eval_R2_CI <- function(data, response, predictors, level=.95) {
    data %>% fit_glm(
        compose_regression_formula_str(response, predictors),
        "gaussian"
    ) -> glm_fit
    # get the p-value
    glm_fit %>% fit_to_df() %>%
    mutate(variable = str_replace_all(variable, "`", "")) %>%
    filter(variable %in% predictors) %>%
    pull(P) %>%
    # we extract the smallest p-values across multiple predictors for now
    min() -> P_val
    # compute the r-squared value
    glm_fit %>% glm_fit_to_R2() -> rsq
    # call psychometric::CI.Rsq() to compute confidence interval
    # https://rdrr.io/cran/psychometric/man/CI.Rsq.html
    # https://rdrr.io/cran/psychometric/src/R/CI.Rsq.R
    CI.Rsq(rsq, n=nrow(data), k=length(predictors), level=level) %>%
    # format the resulting table
    dplyr::select(-SErsq) %>% mutate(
        metric = "R2",
        response = response,
        predictors = paste(predictors, collapse = "+"),
        P = P_val,
        n = nrow(data)
    ) %>%
    rename("eval"="Rsq", "l_eval"="LCL", "u_eval"="UCL") %>%
    dplyr::select(response, predictors, metric, `eval`, l_eval, u_eval, P, n)
}

In [16]:
phenos <- c("INI1003063", "INI20030780", "INI30120")
thresholds <- c("0_1","0_03","0_01","0_003","0_001","0_0003","0_0001","3e-05","1e-05")

eval_trait_threshold <- function(p, th) {
  pheno_file <- sprintf("pheno/Afr_train_val_%s.pheno.txt", p)
  prs_file   <- sprintf("prs/%s_%s.sscore", p, th)

  pheno <- read_tsv(pheno_file, show_col_types = FALSE)

  prs <- read_tsv(prs_file, show_col_types = FALSE) %>%
    dplyr::select(all_of(c("#FID", "IID", "SCORE1_AVG"))) %>%
    dplyr::rename(FID = `#FID`)



  df <- inner_join(pheno, prs, by = c("FID", "IID"))

  out <- eval_R2_CI(df, response = "PHENO", predictors = c("SCORE1_AVG"))
  out$pheno_id  <- p
  out$threshold <- th
  out
}

# Correct nesting: pass p from the outer map into the inner map
results <- do.call(rbind, lapply(phenos, function(p)
  do.call(rbind, lapply(thresholds, function(th) eval_trait_threshold(p, th)))
))

In [17]:
results

response,predictors,metric,eval,l_eval,u_eval,P,n,pheno_id,threshold
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
PHENO,SCORE1_AVG,R2,0.007089502,0.002155183,0.01202382,2.193122e-08,4404,INI1003063,0_1
PHENO,SCORE1_AVG,R2,0.006447582,0.001738909,0.01115626,9.511648e-08,4404,INI1003063,0_03
PHENO,SCORE1_AVG,R2,0.007314911,0.002303900,0.01232592,1.311030e-08,4404,INI1003063,0_01
PHENO,SCORE1_AVG,R2,0.008275267,0.002950612,0.01359992,1.468937e-09,4404,INI1003063,0_003
PHENO,SCORE1_AVG,R2,0.008718596,0.003255617,0.01418158,5.355583e-10,4404,INI1003063,0_001
PHENO,SCORE1_AVG,R2,0.008608131,0.003179265,0.01403700,6.885905e-10,4404,INI1003063,0_0003
PHENO,SCORE1_AVG,R2,0.009290545,0.003654477,0.01492661,1.458642e-10,4404,INI1003063,0_0001
PHENO,SCORE1_AVG,R2,0.007630408,0.002514101,0.01274672,6.383850e-09,4404,INI1003063,3e-05
PHENO,SCORE1_AVG,R2,0.007366722,0.002338259,0.01239519,1.164853e-08,4404,INI1003063,1e-05


In [22]:
best_results <- results %>%
  group_by(pheno_id) %>%
  dplyr::slice_max(order_by = eval, n = 1, with_ties = FALSE) %>%
  ungroup()

In [23]:
best_results

response,predictors,metric,eval,l_eval,u_eval,P,n,pheno_id,threshold
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
PHENO,SCORE1_AVG,R2,0.009290545,0.003654477,0.01492661,1.458642e-10,4404,INI1003063,0_0001
PHENO,SCORE1_AVG,R2,0.035797849,0.025204461,0.04639124,6.191473e-38,4550,INI20030780,3e-05
PHENO,SCORE1_AVG,R2,0.013649466,0.006968891,0.02033004,2.401515e-15,4565,INI30120,0_1
